In [2]:
import pandas as pd 
import numpy as np 
import psycopg2

In [3]:
conn = psycopg2.connect(
    dbname="prueba_sephora",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432")

In [4]:
cur = conn.cursor()

In [39]:
df_principal = pd.read_csv('productos_maquillaje.csv')
pd.set_option('display.max_columns', None)
df_principal

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,fecha_extraccion,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,23.0,1,2.0,12,2025-04-15,NaN,NaN,"Mate, Irisado/Nacarado, Brillante",NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Liquido
1,Rostro,Polvos bronceadores,Dior,Dior Forever Nude Bronze,Polvos bronceadores acabado radiante edición l...,65.0,0,0.0,2,2025-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,Ligera,Natural,NaN
2,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,48.0,434,4.2,3,2025-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Aceite
3,Ojos,Paleta de sombras de ojos,Dior,Diorshow 5 Couleurs edición limitada,sombras de ojos de colores intensos,75.0,0,0.0,2,2025-04-15,NaN,NaN,Irisado/Nacarado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,37.0,0,0.0,2,2025-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,Purpurina,NaN


In [40]:
df_principal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   categoria            5 non-null      object 
 1   subcategoria         5 non-null      object 
 2   marca                5 non-null      object 
 3   nombre               5 non-null      object 
 4   descripcion          5 non-null      object 
 5   precio               5 non-null      float64
 6   numero_valoraciones  5 non-null      int64  
 7   valoracion           5 non-null      float64
 8   num_variaciones      5 non-null      int64  
 9   fecha_extraccion     5 non-null      object 
 10  formato              0 non-null      float64
 11  responsabilidad      0 non-null      float64
 12  efecto_sombra        2 non-null      object 
 13  efecto_labios        0 non-null      float64
 14  efecto_mascara       0 non-null      float64
 15  tipo_brocha          0 non-null      float64

In [ ]:
tabla_marcas = pd.DataFrame(df_principal["marca"].unique(), columns=["nombre_marca"])
tabla_marcas

data_to_insert = [[row["nombre_marca"]] for indice, row in tabla_marcas.iterrows()]

insert_query = """
        INSERT INTO marcas (nombre_marca)
        VALUES (%s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

,nombre_marca
0,Rem Beauty
1,Dior
2,Dior Backstage


In [ ]:
tabla_categorias = pd.DataFrame(df_principal["categoria"].unique(), columns=["nombre_categoria"])
data_to_insert = [[row["nombre_categoria"]] for indice, row in tabla_categorias.iterrows()]
insert_query = """
        INSERT INTO categorias (nombre_categoria)
        VALUES (%s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
tabla_subcategorias = pd.DataFrame(df_principal["subcategoria"].unique(), columns=["nombre_subcategoria"])
data_to_insert = [[row["nombre_subcategoria"]] for indice, row in tabla_subcategorias.iterrows()]
insert_query = """
        INSERT INTO subcategorias (nombre_subcategoria)
        VALUES (%s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [13]:
cur.execute("SELECT nombre_marca, id_marca FROM marcas")
marcas_dict = dict(cur.fetchall()) 
marcas_dict

{'Rem Beauty': 1, 'Dior': 2, 'Dior Backstage': 3}

In [14]:
cur.execute("SELECT nombre_categoria, id_categoria FROM categorias")
categorias_dict = dict(cur.fetchall()) 
categorias_dict

{'Ojos': 1, 'Rostro': 2, 'Labios': 3, 'Uñas': 4}

In [15]:
cur.execute("SELECT nombre_subcategoria, id_subcategoria FROM subcategorias")
subcategorias_dict = dict(cur.fetchall()) 
subcategorias_dict

{'Sombras de ojos': 1,
 'Polvos bronceadores': 2,
 'Voluminizador labios': 3,
 'Paleta de sombras de ojos': 4,
 'Esmalte de uñas': 5}

In [ ]:
data_to_insert = []
df_productos = df_principal[["nombre", "descripcion", "marca", "categoria", "subcategoria"]]
for _, row in df_productos.iterrows(): 
    nombre = row["nombre"]
    descripcion = row["descripcion"]
    id_marca = marcas_dict.get(row["marca"]) if row["marca"] in marcas_dict else None
    id_categoria = categorias_dict.get(row["categoria"]) if row["categoria"] in categorias_dict else None
    id_subcategoria = subcategorias_dict.get(row["subcategoria"]) if row["subcategoria"] in subcategorias_dict else None
    data_to_insert.append([nombre, descripcion, id_marca, id_categoria, id_subcategoria]) 

tabla_productos = pd.DataFrame(data_to_insert, columns=["nombre", "descripcion", "id_marca", "id_categoria", "id_subcategoria"])
tabla_productos

,nombre,descripcion,id_marca,id_categoria,id_subcategoria
0,Starlet,Sombra de ojos líquida,1,1,1
1,Dior Forever Nude Bronze,Polvos bronceadores acabado radiante edición l...,2,2,2
2,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,3,3,3
3,Diorshow 5 Couleurs edición limitada,sombras de ojos de colores intensos,2,1,4
4,Dior Vernis,Laca de uñas efecto gel color couture,2,4,5


In [27]:
insert_query = """
        INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
        VALUES (%s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [28]:
cur.execute("SELECT nombre, id_producto FROM productos")
productos_dict = dict(cur.fetchall()) 
productos_dict

{'Starlet': 1,
 'Dior Forever Nude Bronze': 2,
 'Dior Lip Glow Oil': 3,
 'Diorshow 5 Couleurs edición limitada': 4,
 'Dior Vernis': 5}

In [30]:
data_to_insert = []
df_historico = df_principal[["nombre", "fecha_extraccion", "precio", "numero_valoraciones", "valoracion", "num_variaciones"]]
for _, row in df_historico.iterrows(): 
    id_producto = productos_dict.get(row["nombre"]) if row["nombre"] in productos_dict else None
    fecha_extraccion = row["fecha_extraccion"]
    precio = row["precio"]
    numero_valoraciones = row["numero_valoraciones"]
    valoracion = row["valoracion"]
    numero_variaciones = row["num_variaciones"]
    data_to_insert.append([id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones]) 

tabla_historico = pd.DataFrame(data_to_insert, columns=["id_producto", "fecha_extraccion", "precio", "numero_valoraciones", "valoracion", "numero_variaciones"])
tabla_historico

,id_producto,fecha_extraccion,precio,numero_valoraciones,valoracion,numero_variaciones
0,1,2025-04-15,23.0,1,2.0,12
1,2,2025-04-15,65.0,0,0.0,2
2,3,2025-04-15,48.0,434,4.2,3
3,4,2025-04-15,75.0,0,0.0,2
4,5,2025-04-15,37.0,0,0.0,2


In [31]:
insert_query = """
        INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
        VALUES (%s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

TABLAS FILTROS

In [33]:
df_efectos_sombra = df_principal["efecto_sombra"].dropna()
efectos_sombra_list = df_efectos_sombra.apply(lambda x: [ef.strip() for ef in x.split(",")])
efectos_sombra_unicos = set([efecto for sublist in efectos_sombra_list for efecto in sublist])

tabla_efectos_sombra = pd.DataFrame(efectos_sombra_unicos, columns=["nombre_efecto"])

data_to_insert = [[row["nombre_efecto"]] for _, row in tabla_efectos_sombra.iterrows()]

tabla_efectos_sombra

,nombre_efecto
0,Brillante
1,Irisado/Nacarado
2,Mate


In [34]:
insert_query = """
    INSERT INTO efectos_sombra (nombre_efecto)
    VALUES (%s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [37]:
cur.execute("SELECT nombre_efecto, id_efecto_sombra FROM efectos_sombra")
efectos_sombra_dict = dict(cur.fetchall()) 
efectos_sombra_dict

{'Brillante': 1, 'Irisado/Nacarado': 2, 'Mate': 3}

In [38]:
data_to_insert = []
for _, row in df_principal.iterrows():
    if pd.isna(row["efecto_sombra"]):
        continue

    id_producto = productos_dict.get(row["nombre"])

    efectos_sombra = [ef.strip() for ef in row["efecto_sombra"].split(",")]
    
    for efecto in efectos_sombra:
        id_efecto_sombra = efectos_sombra_dict.get(efecto)
        if id_producto and id_efecto_sombra:
            data_to_insert.append((id_producto, id_efecto_sombra))

insert_query = """
    INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra)
    VALUES (%s, %s)
"""  

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [35]:
df_texturas = df_principal["textura"].dropna()
texturas_list = df_texturas.apply(lambda x: [tx.strip() for tx in x.split(",")])
texturas_unicos = set([textura for sublist in texturas_list for textura in sublist])

tabla_texturas = pd.DataFrame(texturas_unicos, columns=["nombre_textura"])

data_to_insert = [[row["nombre_textura"]] for _, row in tabla_texturas.iterrows()]

tabla_texturas

,nombre_textura
0,Liquido
1,Aceite


In [36]:
insert_query = """
    INSERT INTO texturas (nombre_textura)
    VALUES (%s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [41]:
cur.execute("SELECT nombre_textura, id_textura FROM texturas")
texturas_dict = dict(cur.fetchall()) 
texturas_dict

{'Liquido': 1, 'Aceite': 2}

In [42]:
data_to_insert = []
for _, row in df_principal.iterrows():
    if pd.isna(row["textura"]):
        continue

    id_producto = productos_dict.get(row["nombre"])

    texturas = [tx.strip() for tx in row["textura"].split(",")]
    
    for textura in texturas:
        id_textura = texturas_dict.get(textura)
        if id_producto and id_textura:
            data_to_insert.append((id_producto, id_textura))

insert_query = """
    INSERT INTO producto_textura (id_producto, id_textura)
    VALUES (%s, %s)
"""  

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
import pandas as pd
import psycopg2

def conex_bd_creacion_cursor(nombre_db, usuario, contraseña, servidor, puerto):
    conn = psycopg2.connect(
        dbname=nombre_db,
        user=usuario,
        password=contraseña,
        host=servidor,
        port=puerto)
    cur = conn.cursor()
    return conn, cur

def insercion_db(conn, cur, insert_query, data_to_insert):
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

def creacion_tabla_marcas(conn, cur, dataframe):
    tabla_marcas = pd.DataFrame(dataframe["marca"].unique(), columns=["nombre_marca"])
    data_to_insert = [[row["nombre_marca"]] for _, row in tabla_marcas.iterrows()]
    insert_query = """
        INSERT INTO marcas (nombre_marca)
        VALUES (%s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_categorias(conn, cur, dataframe):
    tabla_categorias = pd.DataFrame(dataframe["categoria"].unique(), columns=["nombre_categoria"])
    data_to_insert = [[row["nombre_categoria"]] for _, row in tabla_categorias.iterrows()]
    insert_query = """
        INSERT INTO categorias (nombre_categoria)
        VALUES (%s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_subcategorias(conn, cur, dataframe):
    tabla_subcategorias = pd.DataFrame(dataframe["subcategoria"].unique(), columns=["nombre_subcategoria"])
    data_to_insert = [[row["nombre_subcategoria"]] for _, row in tabla_subcategorias.iterrows()]
    insert_query = """
        INSERT INTO subcategorias (nombre_subcategoria)
        VALUES (%s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def obtener_dict(conn, cur, table_name, column_name, id_column):
    cur.execute(f"SELECT {column_name}, {id_column} FROM {table_name}")
    return dict(cur.fetchall())

def creacion_tabla_productos(conn, cur, dataframe, marcas_dict, categorias_dict, subcategorias_dict):
    data_to_insert = []
    df_productos = dataframe[["nombre", "descripcion", "marca", "categoria", "subcategoria"]]
    for _, row in df_productos.iterrows():
        nombre = row["nombre"]
        descripcion = row["descripcion"]
        id_marca = marcas_dict.get(row["marca"])
        id_categoria = categorias_dict.get(row["categoria"])
        id_subcategoria = subcategorias_dict.get(row["subcategoria"])
        data_to_insert.append([nombre, descripcion, id_marca, id_categoria, id_subcategoria])

    insert_query = """
        INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
        VALUES (%s, %s, %s, %s, %s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_historico(conn, cur, dataframe, productos_dict):
    data_to_insert = []
    df_historico = dataframe[["nombre", "fecha_extraccion", "precio", "numero_valoraciones", "valoracion", "num_variaciones"]]
    for _, row in df_historico.iterrows():
        id_producto = productos_dict.get(row["nombre"])
        fecha_extraccion = row["fecha_extraccion"]
        precio = row["precio"]
        numero_valoraciones = row["numero_valoraciones"]
        valoracion = row["valoracion"]
        numero_variaciones = row["num_variaciones"]
        data_to_insert.append([id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones])

    insert_query = """
        INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_efectos_sombra(conn, cur, dataframe):
    df_efectos_sombra = dataframe["efecto_sombra"].dropna()
    efectos_sombra_list = df_efectos_sombra.apply(lambda x: [ef.strip() for ef in x.split(",")])
    efectos_sombra_unicos = set([efecto for sublist in efectos_sombra_list for efecto in sublist])

    tabla_efectos_sombra = pd.DataFrame(efectos_sombra_unicos, columns=["nombre_efecto"])
    data_to_insert = [[row["nombre_efecto"]] for _, row in tabla_efectos_sombra.iterrows()]
    insert_query = """
        INSERT INTO efectos_sombra (nombre_efecto)
        VALUES (%s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_texturas(conn, cur, dataframe):
    df_texturas = dataframe["textura"].dropna()
    texturas_list = df_texturas.apply(lambda x: [tx.strip() for tx in x.split(",")])
    texturas_unicos = set([textura for sublist in texturas_list for textura in sublist])

    tabla_texturas = pd.DataFrame(texturas_unicos, columns=["nombre_textura"])
    data_to_insert = [[row["nombre_textura"]] for _, row in tabla_texturas.iterrows()]
    insert_query = """
        INSERT INTO texturas (nombre_textura)
        VALUES (%s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_producto_efecto_sombra(conn, cur, dataframe, productos_dict, efectos_sombra_dict):
    data_to_insert = []
    for _, row in dataframe.iterrows():
        if pd.isna(row["efecto_sombra"]):
            continue

        id_producto = productos_dict.get(row["nombre"])
        efectos_sombra = [ef.strip() for ef in row["efecto_sombra"].split(",")]

        for efecto in efectos_sombra:
            id_efecto_sombra = efectos_sombra_dict.get(efecto)
            if id_producto and id_efecto_sombra:
                data_to_insert.append((id_producto, id_efecto_sombra))

    insert_query = """
        INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra)
        VALUES (%s, %s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def creacion_tabla_producto_textura(conn, cur, dataframe, productos_dict, texturas_dict):
    data_to_insert = []
    for _, row in dataframe.iterrows():
        if pd.isna(row["textura"]):
            continue

        id_producto = productos_dict.get(row["nombre"])
        texturas = [tx.strip() for tx in row["textura"].split(",")]

        for textura in texturas:
            id_textura = texturas_dict.get(textura)
            if id_producto and id_textura:
                data_to_insert.append((id_producto, id_textura))

    insert_query = """
        INSERT INTO producto_textura (id_producto, id_textura)
        VALUES (%s, %s)
    """
    insercion_db(conn, cur, insert_query, data_to_insert)

def crear_tablas(nombre_db, usuario, contraseña, servidor, puerto, df_principal):
    try:
        # establece conexion y crea cursor
        conn, cur = conex_bd_creacion_cursor(nombre_db, usuario, contraseña, servidor, puerto)

        # crea las tablas
        creacion_tabla_marcas(conn, cur, df_principal)
        creacion_tabla_categorias(conn, cur, df_principal)
        creacion_tabla_subcategorias(conn, cur, df_principal)

        # obtener los diccionarios para las relaciones
        marcas_dict = obtener_dict(conn, cur, "marcas", "nombre_marca", "id_marca")
        categorias_dict = obtener_dict(conn, cur, "categorias", "nombre_categoria", "id_categoria")
        subcategorias_dict = obtener_dict(conn, cur, "subcategorias", "nombre_subcategoria", "id_subcategoria")

        creacion_tabla_productos(conn, cur, df_principal, marcas_dict, categorias_dict, subcategorias_dict)

        productos_dict = obtener_dict(conn, cur, "productos", "nombre", "id_producto")
        creacion_tabla_historico(conn, cur, df_principal, productos_dict)

        creacion_tabla_efectos_sombra(conn, cur, df_principal)
        efectos_sombra_dict = obtener_dict(conn, cur, "efectos_sombra", "nombre_efecto", "id_efecto_sombra")
        creacion_tabla_producto_efecto_sombra(conn, cur, df_principal, productos_dict, efectos_sombra_dict)

        creacion_tabla_texturas(conn, cur, df_principal)
        texturas_dict = obtener_dict(conn, cur, "texturas", "nombre_textura", "id_textura")
        creacion_tabla_producto_textura(conn, cur, df_principal, productos_dict, texturas_dict)

    except Exception as e:
        if conn:
            conn.close()
        if cur:
            cur.close()
        raise Exception(f"Error en el proceso de carga de datos: {e}")
    else:
        cur.close()
        conn.close()

In [ ]:
def insertar_producto_y_historico(conn, cur, df_productos):
    # Iterar sobre cada producto en el DataFrame
    for index, row in df_productos.iterrows():
        nombre_producto = row['nombre']
        descripcion = row['descripcion']
        marca = row['marca']
        categoria = row['categoria']
        subcategoria = row['subcategoria']
        precio = row['precio']
        valoraciones = row['numero_valoraciones']
        variaciones = row['num_variaciones']
        valoracion_media = row['valoracion']
        fecha_extraccion = row['fecha_extraccion']
        
        # Filtros
        formato = row['formato']
        responsabilidad = row['responsabilidad']
        efecto_sombra = row['efecto_sombra']
        efecto_labios = row['efecto_labios']
        efecto_mascara = row['efecto_mascara']
        tipo_piel = row['tipo_piel']
        cobertura = row['cobertura']
        acabado = row['acabado']
        textura = row['textura']
        
        # Comprobar si el producto ya existe en la base de datos
        cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre_producto,))
        producto_existente = cur.fetchone()
        
        if producto_existente:
            # Caso 1: El producto ya existe, solo insertar en la tabla historico
            id_producto = producto_existente[0]
            cur.execute("""
                INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (id_producto, fecha_extraccion, precio, valoraciones, valoracion_media, variaciones))
            print(f"Histórico insertado para el producto existente: {nombre_producto}")
        
        else:
            # Caso 2: El producto no existe, insertamos todo (marca, categoría, subcategoría, producto, filtros, historico)
            # 1. Comprobar si la marca ya existe
            cur.execute("SELECT id_marca FROM marcas WHERE nombre_marca = %s", (marca,))
            marca_result = cur.fetchone()
            if marca_result:
                id_marca = marca_result[0]
            else:
                # Insertar nueva marca
                cur.execute("INSERT INTO marcas (nombre_marca) VALUES (%s) RETURNING id_marca", (marca,))
                id_marca = cur.fetchone()[0]
            
            # 2. Comprobar si la categoría ya existe
            cur.execute("SELECT id_categoria FROM categorias WHERE nombre_categoria = %s", (categoria,))
            categoria_result = cur.fetchone()
            if categoria_result:
                id_categoria = categoria_result[0]
            else:
                # Insertar nueva categoría
                cur.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s) RETURNING id_categoria", (categoria,))
                id_categoria = cur.fetchone()[0]
            
            # 3. Comprobar si la subcategoría ya existe
            cur.execute("SELECT id_subcategoria FROM subcategorias WHERE nombre_subcategoria = %s", (subcategoria,))
            subcategoria_result = cur.fetchone()
            if subcategoria_result:
                id_subcategoria = subcategoria_result[0]
            else:
                # Insertar nueva subcategoría
                cur.execute("INSERT INTO subcategorias (nombre_subcategoria) VALUES (%s) RETURNING id_subcategoria", (subcategoria,))
                id_subcategoria = cur.fetchone()[0]
            
            # 4. Insertar el nuevo producto
            cur.execute("""
                INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
                VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
            """, (nombre_producto, descripcion, id_marca, id_categoria, id_subcategoria))
            id_producto = cur.fetchone()[0]
            
            # 5. Insertar el registro en la tabla historico
            cur.execute("""
                INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (id_producto, fecha_extraccion, precio, valoraciones, valoracion_media, variaciones))
            
            # 6. Insertar los filtros en las tablas correspondientes
            if pd.notna(formato):
                cur.execute("SELECT id_formato FROM formatos WHERE nombre_formato = %s", (formato,))
                formato_result = cur.fetchone()
                if not formato_result:
                    cur.execute("INSERT INTO formatos (nombre_formato) VALUES (%s) RETURNING id_formato", (formato,))
                    id_formato = cur.fetchone()[0]
                else:
                    id_formato = formato_result[0]
                cur.execute("INSERT INTO producto_formato (id_producto, id_formato) VALUES (%s, %s)", (id_producto, id_formato))
            
            if pd.notna(responsabilidad):
                cur.execute("SELECT id_responsabilidad FROM responsabilidades WHERE nombre_responsabilidad = %s", (responsabilidad,))
                responsabilidad_result = cur.fetchone()
                if not responsabilidad_result:
                    cur.execute("INSERT INTO responsabilidades (nombre_responsabilidad) VALUES (%s) RETURNING id_responsabilidad", (responsabilidad,))
                    id_responsabilidad = cur.fetchone()[0]
                else:
                    id_responsabilidad = responsabilidad_result[0]
                cur.execute("INSERT INTO producto_responsabilidad (id_producto, id_responsabilidad) VALUES (%s, %s)", (id_producto, id_responsabilidad))
            
            if pd.notna(efecto_sombra):
                cur.execute("SELECT id_efecto_sombra FROM efectos_sombra WHERE nombre_efecto = %s", (efecto_sombra,))
                efecto_sombra_result = cur.fetchone()
                if not efecto_sombra_result:
                    cur.execute("INSERT INTO efectos_sombra (nombre_efecto) VALUES (%s) RETURNING id_efecto_sombra", (efecto_sombra,))
                    id_efecto_sombra = cur.fetchone()[0]
                else:
                    id_efecto_sombra = efecto_sombra_result[0]
                cur.execute("INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra) VALUES (%s, %s)", (id_producto, id_efecto_sombra))
            
            # Repetir el proceso similar para los otros filtros (efecto_labios, efecto_mascara, tipo_piel, cobertura, acabado, textura)
            # Similar a lo que hemos hecho para los otros filtros.
            
            # 7. Confirmar la transacción
            conn.commit()
            print(f"Producto {nombre_producto} insertado y actualizado en historico.")


In [43]:
df_segundo = pd.read_csv('productos_maquillaje_2025-04-19.csv')
pd.set_option('display.max_columns', None)
df_segundo

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,fecha_extraccion,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,23.00,1,2.0,12,2025-04-19,NaN,Clean at Sephora,"Mate, Irisado/Nacarado, Brillante",NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Liquido
1,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,48.00,437,4.2,3,2025-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Aceite
2,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,37.00,0,0.0,2,2025-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,Purpurina,NaN
3,Labios,Gloss/ Brillo labial,Valentino Make Up,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...",39.99,5,4.6,5,2025-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Liquido
4,Labios,Gloss/ Brillo labial,Fenty Beauty,Gloss Bomb Oil,Aceite-gloss para los labios,28.00,0,0.0,7,2025-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Liquido


In [44]:
for index, row in df_segundo.iterrows():
        nombre = row['nombre']
        descripcion = row['descripcion']
        marca = row['marca']
        categoria = row['categoria']
        subcategoria = row['subcategoria']
        precio = row['precio']
        numero_valoraciones = row['numero_valoraciones']
        num_variaciones = row['num_variaciones']
        valoracion = row['valoracion']
        fecha_extraccion = row['fecha_extraccion']
        formato = row['formato']
        responsabilidad = row['responsabilidad']
        efecto_sombra = row['efecto_sombra']
        efecto_labios = row['efecto_labios']
        efecto_mascara = row['efecto_mascara']
        tipo_piel = row['tipo_piel']
        cobertura = row['cobertura']
        acabado = row['acabado']
        textura = row['textura']
        
        # Comprobar si el producto ya existe en la base de datos
        cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre,))
        producto_existente = cur.fetchone()
        
        if producto_existente:
            # Caso 1: El producto ya existe, solo insertar en la tabla historico
            id_producto = producto_existente[0]
            cur.execute("""
                INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
            print(f"Histórico insertado para el producto existente: {nombre}")
        
        else:
            # Caso 2: El producto no existe, insertamos todo (marca, categoría, subcategoría, producto, filtros, historico)
            # 1. Comprobar si la marca ya existe
            cur.execute("SELECT id_marca FROM marcas WHERE nombre_marca = %s", (marca,))
            marca_result = cur.fetchone()
            if marca_result:
                id_marca = marca_result[0]
            else:
                # Insertar nueva marca
                cur.execute("INSERT INTO marcas (nombre_marca) VALUES (%s) RETURNING id_marca", (marca,))
                id_marca = cur.fetchone()[0]
            
            # 2. Comprobar si la categoría ya existe
            cur.execute("SELECT id_categoria FROM categorias WHERE nombre_categoria = %s", (categoria,))
            categoria_result = cur.fetchone()
            if categoria_result:
                id_categoria = categoria_result[0]
            else:
                # Insertar nueva categoría
                cur.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s) RETURNING id_categoria", (categoria,))
                id_categoria = cur.fetchone()[0]
            
            # 3. Comprobar si la subcategoría ya existe
            cur.execute("SELECT id_subcategoria FROM subcategorias WHERE nombre_subcategoria = %s", (subcategoria,))
            subcategoria_result = cur.fetchone()
            if subcategoria_result:
                id_subcategoria = subcategoria_result[0]
            else:
                # Insertar nueva subcategoría
                cur.execute("INSERT INTO subcategorias (nombre_subcategoria) VALUES (%s) RETURNING id_subcategoria", (subcategoria,))
                id_subcategoria = cur.fetchone()[0]
            
            # 4. Insertar el nuevo producto
            cur.execute("""
                INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
                VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
            """, (nombre, descripcion, id_marca, id_categoria, id_subcategoria))
            id_producto = cur.fetchone()[0]
            
            # 5. Insertar el registro en la tabla historico
            cur.execute("""
                INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
            
            # 6. Insertar los filtros en las tablas correspondientes            
            if pd.notna(efecto_sombra):
                cur.execute("SELECT id_efecto_sombra FROM efectos_sombra WHERE nombre_efecto = %s", (efecto_sombra,))
                efecto_sombra_result = cur.fetchone()
                if not efecto_sombra_result:
                    cur.execute("INSERT INTO efectos_sombra (nombre_efecto) VALUES (%s) RETURNING id_efecto_sombra", (efecto_sombra,))
                    id_efecto_sombra = cur.fetchone()[0]
                else:
                    id_efecto_sombra = efecto_sombra_result[0]
                cur.execute("INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra) VALUES (%s, %s)", (id_producto, id_efecto_sombra))

            if pd.notna(textura):
                cur.execute("SELECT id_textura FROM texturas WHERE nombre_textura = %s", (textura,))
                textura_result = cur.fetchone()
                if not textura_result:
                    cur.execute("INSERT INTO texturas (nombre_textura) VALUES (%s) RETURNING id_textura", (textura,))
                    id_textura = cur.fetchone()[0]
                else:
                    id_textura = textura_result[0]
                cur.execute("INSERT INTO producto_textura (id_producto, id_textura) VALUES (%s, %s)", (id_producto, id_textura))      

            # Similar a lo que hemos hecho para los otros filtros.
            
            # 7. Confirmar la transacción
            conn.commit()
            print(f"Producto {nombre} insertado y actualizado en historico.")

Histórico insertado para el producto existente: Starlet
Histórico insertado para el producto existente: Dior Lip Glow Oil
Histórico insertado para el producto existente: Dior Vernis
Producto Puffer Gloss insertado y actualizado en historico.
Producto Gloss Bomb Oil insertado y actualizado en historico.


In [48]:
# codigo completo de insercion y actualizacion de datos tras los scrapeos

import pandas as pd
import psycopg2
import numpy as np

# Conexión
conn = psycopg2.connect(
    dbname="prueba_sephora",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Leer CSV
df = pd.read_csv("productos_maquillaje_2025-04-19.csv")
pd.set_option('display.max_columns', None)

for _, row in df.iterrows():
    nombre = row['nombre']
    descripcion = row['descripcion']
    marca = row['marca']
    categoria = row['categoria']
    subcategoria = row['subcategoria']
    precio = row['precio']
    numero_valoraciones = row['numero_valoraciones']
    num_variaciones = row['num_variaciones']
    valoracion = row['valoracion']
    fecha_extraccion = row['fecha_extraccion']
    
    efecto_sombra = row.get('efecto_sombra')
    textura = row.get('textura')

    # 1. ¿Existe el producto?
    cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre,))
    producto_result = cur.fetchone()

    if producto_result:
        id_producto = producto_result[0]
        # Solo insertar histórico
        cur.execute("""
            INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
        print(f"Histórico actualizado para producto existente: {nombre}")
    else:
        # 2. Insertar Marca si no existe
        cur.execute("SELECT id_marca FROM marcas WHERE nombre_marca = %s", (marca,))
        marca_result = cur.fetchone()
        if marca_result:
            id_marca = marca_result[0]
        else:
            cur.execute("INSERT INTO marcas (nombre_marca) VALUES (%s) RETURNING id_marca", (marca,))
            id_marca = cur.fetchone()[0]

        # 3. Insertar Categoría si no existe
        cur.execute("SELECT id_categoria FROM categorias WHERE nombre_categoria = %s", (categoria,))
        categoria_result = cur.fetchone()
        if categoria_result:
            id_categoria = categoria_result[0]
        else:
            cur.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s) RETURNING id_categoria", (categoria,))
            id_categoria = cur.fetchone()[0]

        # 4. Insertar Subcategoría si no existe
        cur.execute("SELECT id_subcategoria FROM subcategorias WHERE nombre_subcategoria = %s", (subcategoria,))
        subcategoria_result = cur.fetchone()
        if subcategoria_result:
            id_subcategoria = subcategoria_result[0]
        else:
            cur.execute("INSERT INTO subcategorias (nombre_subcategoria) VALUES (%s) RETURNING id_subcategoria", (subcategoria,))
            id_subcategoria = cur.fetchone()[0]

        # 5. Insertar Producto
        cur.execute("""
            INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
            VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
        """, (nombre, descripcion, id_marca, id_categoria, id_subcategoria))
        id_producto = cur.fetchone()[0]

        # 6. Insertar Histórico
        cur.execute("""
            INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
        
        # 7. Insertar Filtros si existen
        # Efecto sombra
        if pd.notna(efecto_sombra):
            efectos = [e.strip() for e in efecto_sombra.split(",")]
            for efecto in efectos:
                cur.execute("SELECT id_efecto_sombra FROM efectos_sombra WHERE nombre_efecto = %s", (efecto,))
                efecto_result = cur.fetchone()
                if not efecto_result:
                    cur.execute("INSERT INTO efectos_sombra (nombre_efecto) VALUES (%s) RETURNING id_efecto_sombra", (efecto,))
                    id_efecto = cur.fetchone()[0]
                else:
                    id_efecto = efecto_result[0]
                cur.execute("INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra) VALUES (%s, %s)", (id_producto, id_efecto))

        # Textura
        if pd.notna(textura):
            cur.execute("SELECT id_textura FROM texturas WHERE nombre_textura = %s", (textura,))
            textura_result = cur.fetchone()
            if not textura_result:
                cur.execute("INSERT INTO texturas (nombre_textura) VALUES (%s) RETURNING id_textura", (textura,))
                id_textura = cur.fetchone()[0]
            else:
                id_textura = textura_result[0]
            cur.execute("INSERT INTO producto_textura (id_producto, id_textura) VALUES (%s, %s)", (id_producto, id_textura))

        print(f"Producto nuevo insertado: {nombre}")

    conn.commit()

# Cerrar conexión
cur.close()
conn.close()
print("Proceso finalizado ✅")

Histórico actualizado para producto existente: Starlet
Histórico actualizado para producto existente: Dior Lip Glow Oil
Histórico actualizado para producto existente: Dior Vernis
Producto nuevo insertado: Puffer Gloss
Producto nuevo insertado: Gloss Bomb Oil
Proceso finalizado ✅


In [ ]:
# codigo completo de insercion y actualizacion de datos tras los scrapeos

import pandas as pd
import psycopg2
import numpy as np

# Conexión
conn = psycopg2.connect(
    dbname="prueba_sephora",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Leer CSV
df = pd.read_csv("productos_maquillaje.csv")
pd.set_option('display.max_columns', None)

for _, row in df.iterrows():
    nombre = row['nombre']
    descripcion = row['descripcion']
    marca = row['marca']
    categoria = row['categoria']
    subcategoria = row['subcategoria']
    precio = row['precio']
    numero_valoraciones = row['numero_valoraciones']
    num_variaciones = row['num_variaciones']
    valoracion = row['valoracion']
    fecha_extraccion = row['fecha_extraccion']
    
    efecto_sombra = row.get('efecto_sombra')
    textura = row.get('textura')
    formato = row.get('formato')
    responsabilidad = row.get('responsabilidad')
    efecto_labios = row.get('efecto_labios')
    efecto_mascara = row.get('efecto_mascara')
    tipo_brocha = row.get('tipo_brocha')
    formulacion = row.get('formulacion')
    tipo_piel = row.get('tipo_piel')
    cobertura = row.get('cobertura')
    acabado = row.get('acabado')

    # 1. ¿Existe el producto?
    cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre,))
    producto_result = cur.fetchone()

    if producto_result:
        id_producto = producto_result[0]
        # Solo insertar histórico
        cur.execute("""
            INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
        print(f"Histórico actualizado para producto existente: {nombre}")
    else:
        # 2. Insertar Marca si no existe
        cur.execute("SELECT id_marca FROM marcas WHERE nombre_marca = %s", (marca,))
        marca_result = cur.fetchone()
        if marca_result:
            id_marca = marca_result[0]
        else:
            cur.execute("INSERT INTO marcas (nombre_marca) VALUES (%s) RETURNING id_marca", (marca,))
            id_marca = cur.fetchone()[0]

        # 3. Insertar Categoría si no existe
        cur.execute("SELECT id_categoria FROM categorias WHERE nombre_categoria = %s", (categoria,))
        categoria_result = cur.fetchone()
        if categoria_result:
            id_categoria = categoria_result[0]
        else:
            cur.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s) RETURNING id_categoria", (categoria,))
            id_categoria = cur.fetchone()[0]

        # 4. Insertar Subcategoría si no existe
        cur.execute("SELECT id_subcategoria FROM subcategorias WHERE nombre_subcategoria = %s", (subcategoria,))
        subcategoria_result = cur.fetchone()
        if subcategoria_result:
            id_subcategoria = subcategoria_result[0]
        else:
            cur.execute("INSERT INTO subcategorias (nombre_subcategoria) VALUES (%s) RETURNING id_subcategoria", (subcategoria,))
            id_subcategoria = cur.fetchone()[0]

        # 5. Insertar Producto
        cur.execute("""
            INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
            VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
        """, (nombre, descripcion, id_marca, id_categoria, id_subcategoria))
        id_producto = cur.fetchone()[0]

        # 6. Insertar Histórico
        cur.execute("""
            INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))
        
        # 7. Insertar Filtros si existen
        # Efecto sombra
        if pd.notna(efecto_sombra):
            efectos = [e.strip() for e in efecto_sombra.split(",")]
            for efecto in efectos:
                cur.execute("SELECT id_efecto_sombra FROM efectos_sombra WHERE nombre_efecto = %s", (efecto,))
                efecto_result = cur.fetchone()
                if not efecto_result:
                    cur.execute("INSERT INTO efectos_sombra (nombre_efecto) VALUES (%s) RETURNING id_efecto_sombra", (efecto,))
                    id_efecto = cur.fetchone()[0]
                else:
                    id_efecto = efecto_result[0]
                cur.execute("INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra) VALUES (%s, %s)", (id_producto, id_efecto))

        # Textura
        if pd.notna(textura):
            texturas = [t.strip() for t in textura.split(",")]
            for t in texturas:
                cur.execute("SELECT id_textura FROM texturas WHERE nombre_textura = %s", (t,))
                textura_result = cur.fetchone()
                if not textura_result:
                    cur.execute("INSERT INTO texturas (nombre_textura) VALUES (%s) RETURNING id_textura", (t,))
                    id_textura = cur.fetchone()[0]
                else:
                    id_textura = textura_result[0]
                cur.execute("INSERT INTO producto_textura (id_producto, id_textura) VALUES (%s, %s)", (id_producto, id_textura))

        # Formato
        if pd.notna(formato):
            formatos = [f.strip() for f in formato.split(",")]
            for f in formatos:
                cur.execute("SELECT id_formato FROM formatos WHERE nombre_formato = %s", (f,))
                formato_result = cur.fetchone()
                if not formato_result:
                    cur.execute("INSERT INTO formatos (nombre_formato) VALUES (%s) RETURNING id_formato", (f,))
                    id_formato = cur.fetchone()[0]
                else:
                    id_formato = formato_result[0]
                cur.execute("INSERT INTO producto_formato (id_producto, id_formato) VALUES (%s, %s)", (id_producto, id_formato))

        # Responsabilidad
        if pd.notna(responsabilidad):
            responsabilidades = [r.strip() for r in responsabilidad.split(",")]
            for r in responsabilidades:
                cur.execute("SELECT id_responsabilidad FROM responsabilidades WHERE nombre_responsabilidad = %s", (r,))
                responsabilidad_result = cur.fetchone()
                if not responsabilidad_result:
                    cur.execute("INSERT INTO responsabilidades (nombre_responsabilidad) VALUES (%s) RETURNING id_responsabilidad", (r,))
                    id_responsabilidad = cur.fetchone()[0]
                else:
                    id_responsabilidad = responsabilidad_result[0]
                cur.execute("INSERT INTO producto_responsabilidad (id_producto, id_responsabilidad) VALUES (%s, %s)", (id_producto, id_responsabilidad))

        # Efecto Labios
        if pd.notna(efecto_labios):
            efectos_labios = [e.strip() for e in efecto_labios.split(",")]
            for e in efectos_labios:
                cur.execute("SELECT id_efecto_labios FROM efectos_labios WHERE nombre_efecto = %s", (e,))
                efecto_result = cur.fetchone()
                if not efecto_result:
                    cur.execute("INSERT INTO efectos_labios (nombre_efecto) VALUES (%s) RETURNING id_efecto_labios", (e,))
                    id_efecto = cur.fetchone()[0]
                else:
                    id_efecto = efecto_result[0]
                cur.execute("INSERT INTO producto_efecto_labios (id_producto, id_efecto_labios) VALUES (%s, %s)", (id_producto, id_efecto))

        # Efecto Mascara
        if pd.notna(efecto_mascara):
            efectos_mascara = [e.strip() for e in efecto_mascara.split(",")]
            for e in efectos_mascara:
                cur.execute("SELECT id_efecto_mascara FROM efectos_mascara WHERE nombre_efecto = %s", (e,))
                efecto_result = cur.fetchone()
                if not efecto_result:
                    cur.execute("INSERT INTO efectos_mascara (nombre_efecto) VALUES (%s) RETURNING id_efecto_mascara", (e,))
                    id_efecto = cur.fetchone()[0]
                else:
                    id_efecto = efecto_result[0]
                cur.execute("INSERT INTO producto_efecto_mascara (id_producto, id_efecto_mascara) VALUES (%s, %s)", (id_producto, id_efecto))

        # Formulación
        if pd.notna(formulacion):
            formulaciones = [f.strip() for f in formulacion.split(",")]
            for f in formulaciones:
                cur.execute("SELECT id_formulacion FROM formulaciones WHERE nombre_formulacion = %s", (f,))
                formulacion_result = cur.fetchone()
                if not formulacion_result:
                    cur.execute("INSERT INTO formulaciones (nombre_formulacion) VALUES (%s) RETURNING id_formulacion", (f,))
                    id_formulacion = cur.fetchone()[0]
                else:
                    id_formulacion = formulacion_result[0]
                cur.execute("INSERT INTO producto_formulacion (id_producto, id_formulacion) VALUES (%s, %s)", (id_producto, id_formulacion))

        # Tipo Piel
        if pd.notna(tipo_piel):
            tipos_piel = [t.strip() for t in tipo_piel.split(",")]
            for t in tipos_piel:
                cur.execute("SELECT id_tipo_piel FROM tipos_piel WHERE nombre_tipo_piel = %s", (t,))
                tipo_result = cur.fetchone()
                if not tipo_result:
                    cur.execute("INSERT INTO tipos_piel (nombre_tipo_piel) VALUES (%s) RETURNING id_tipo_piel", (t,))
                    id_tipo = cur.fetchone()[0]
                else:
                    id_tipo = tipo_result[0]
                cur.execute("INSERT INTO producto_tipo_piel (id_producto, id_tipo_piel) VALUES (%s, %s)", (id_producto, id_tipo))

        # Cobertura
        if pd.notna(cobertura):
            coberturas = [c.strip() for c in cobertura.split(",")]
            for c in coberturas:
                cur.execute("SELECT id_cobertura FROM coberturas WHERE nombre_cobertura = %s", (c,))
                cobertura_result = cur.fetchone()
                if not cobertura_result:
                    cur.execute("INSERT INTO coberturas (nombre_cobertura) VALUES (%s) RETURNING id_cobertura", (c,))
                    id_cobertura = cur.fetchone()[0]
                else:
                    id_cobertura = cobertura_result[0]
                cur.execute("INSERT INTO producto_cobertura (id_producto, id_cobertura) VALUES (%s, %s)", (id_producto, id_cobertura))

        # Acabado
        if pd.notna(acabado):
            acabados = [a.strip() for a in acabado.split(",")]
            for a in acabados:
                cur.execute("SELECT id_acabado FROM acabados WHERE nombre_acabado = %s", (a,))
                acabado_result = cur.fetchone()
                if not acabado_result:
                    cur.execute("INSERT INTO acabados (nombre_acabado) VALUES (%s) RETURNING id_acabado", (a,))
                    id_acabado = cur.fetchone()[0]
                else:
                    id_acabado = acabado_result[0]
                cur.execute("INSERT INTO producto_acabado (id_producto, id_acabado) VALUES (%s, %s)", (id_producto, id_acabado))



        print(f"Producto nuevo insertado: {nombre}")

    conn.commit()

# Cerrar conexión
cur.close()
conn.close()
print("Proceso finalizado ✅")

In [13]:
import pandas as pd
import psycopg2

def conectar_bd():
    conn = psycopg2.connect(
        dbname="SEPHORA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    return conn, conn.cursor()

def insertar_historico(cur, id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones):
    cur.execute("""
        INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))

def obtener_o_insertar_id(cur, id_col, tabla, columna, valor):
    cur.execute(f"SELECT {id_col} FROM {tabla} WHERE {columna} = %s", (valor,))
    resultado = cur.fetchone()
    
    if resultado:
        return resultado[0]
    else:
        cur.execute(
            f"INSERT INTO {tabla} ({columna}) VALUES (%s) RETURNING {id_col}",
            (valor,)
        )
        return cur.fetchone()[0]

def insertar_producto(cur, nombre, descripcion, id_marca, id_categoria, id_subcategoria):
    cur.execute("""
        INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
        VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
    """, (nombre, descripcion, id_marca, id_categoria, id_subcategoria))
    return cur.fetchone()[0]

def insertar_filtro(cur, columna, id_col, tabla, nombre_col, id_producto, tabla_intermedia):
    if pd.notna(columna):
        filtros = [v.strip() for v in columna.split(",")]
        for v in filtros:
            cur.execute(f"SELECT {id_col} FROM {tabla} WHERE {nombre_col} = %s", (v,))
            resultado = cur.fetchone()

            if resultado:
                id_filtro = resultado[0]
            else:
                cur.execute(f"INSERT INTO {tabla} ({nombre_col}) VALUES (%s) RETURNING {id_col}", (v,))
                id_filtro = cur.fetchone()[0]

            cur.execute(f"INSERT INTO {tabla_intermedia} (id_producto, {id_col}) VALUES (%s, %s)", (id_producto, id_filtro))

def procesar_productos(df, cur):

    for _, row in df.iterrows():
        nombre = row['nombre']
        descripcion = row['descripcion']
        marca = row['marca']
        categoria = row['categoria']
        subcategoria = row['subcategoria']
        precio = row['precio']
        numero_valoraciones = row['numero_valoraciones']
        num_variaciones = row['num_variaciones']
        valoracion = row['valoracion']
        fecha_extraccion = row['fecha_extraccion']
        efecto_sombra = row.get('efecto_sombra')
        textura = row.get('textura')
        formato = row.get('formato')
        responsabilidad = row.get('responsabilidad')
        efecto_labios = row.get('efecto_labios')
        efecto_mascara = row.get('efecto_mascara')
        formulacion = row.get('formulacion')
        tipo_piel = row.get('tipo_piel')
        cobertura = row.get('cobertura')
        acabado = row.get('acabado')

        cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre,))
        producto_result = cur.fetchone()

        if producto_result:
            id_producto = producto_result[0]
            insertar_historico(cur, id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones)
            print(f"Histórico actualizado para producto existente: {nombre}")
        else:
            id_marca = obtener_o_insertar_id(cur, "id_marca", "marcas", "nombre_marca", marca)
            id_categoria = obtener_o_insertar_id(cur, "id_categoria", "categorias", "nombre_categoria", categoria)
            id_subcategoria = obtener_o_insertar_id(cur, "id_subcategoria", "subcategorias", "nombre_subcategoria", subcategoria)
            id_producto = insertar_producto(cur, nombre, descripcion, id_marca, id_categoria, id_subcategoria)
            insertar_historico(cur, id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones)
            insertar_filtro(cur, efecto_labios, "id_efecto_labios","efectos_labios", "nombre_efecto", id_producto, "producto_efecto_labios")
            insertar_filtro(cur, efecto_sombra, "id_efecto_sombra", "efectos_sombra", "nombre_efecto", id_producto, "producto_efecto_sombra")
            insertar_filtro(cur, textura, "id_textura", "texturas", "nombre_textura", id_producto, "producto_textura")
            insertar_filtro(cur, formato, "id_formato", "formatos", "nombre_formato", id_producto, "producto_formato")  
            insertar_filtro(cur, responsabilidad, "id_responsabilidad", "responsabilidades", "nombre_responsabilidad", id_producto, "producto_responsabilidad")
            insertar_filtro(cur, efecto_mascara, "id_efecto_mascara", "efectos_mascara", "nombre_efecto", id_producto, "producto_efecto_mascara")
            insertar_filtro(cur, formulacion, "id_formulacion", "formulaciones", "nombre_formulacion", id_producto, "producto_formulacion")
            insertar_filtro(cur, tipo_piel, "id_tipo_piel", "tipos_piel", "nombre_tipo_piel", id_producto, "producto_tipo_piel")
            insertar_filtro(cur, cobertura, "id_cobertura", "coberturas", "nombre_cobertura", id_producto, "producto_cobertura")    
            insertar_filtro(cur, acabado, "id_acabado", "acabados", "nombre_acabado", id_producto, "producto_acabado")

def cargar_datos_sephora():
    df = pd.read_csv("../data/productos_maquillaje.csv")
    conn, cur = conectar_bd()

    procesar_productos(df, cur)  
    conn.commit()  

    cur.close()
    conn.close()
    print("Proceso finalizado ✅")

In [32]:
import pandas as pd
import psycopg2

def conectar_bd():
    conn = psycopg2.connect(
        dbname="SEPHORA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    return conn, conn.cursor()

def insertar_historico(cur, id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones):
    cur.execute("""
        INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones))

def obtener_o_insertar_id(cur, id_col, tabla, columna, valor):
    cur.execute(f"SELECT {id_col} FROM {tabla} WHERE {columna} = %s", (valor,))
    resultado = cur.fetchone()
    
    if resultado:
        return resultado[0]
    else:
        cur.execute(
            f"INSERT INTO {tabla} ({columna}) VALUES (%s) RETURNING {id_col}",
            (valor,)
        )
        return cur.fetchone()[0]

def insertar_producto(cur, nombre, descripcion, id_marca, id_categoria, id_subcategoria):
    cur.execute("""
        INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
        VALUES (%s, %s, %s, %s, %s) RETURNING id_producto
    """, (nombre, descripcion, id_marca, id_categoria, id_subcategoria))
    return cur.fetchone()[0]

def insertar_filtro(cur, columna, id_col, tabla, nombre_col, id_producto, tabla_intermedia):
    if pd.notna(columna):
        filtros = [v.strip() for v in columna.split(",")]
        for v in filtros:
            cur.execute(f"SELECT {id_col} FROM {tabla} WHERE {nombre_col} = %s", (v,))
            resultado = cur.fetchone()

            if resultado:
                id_filtro = resultado[0]
            else:
                cur.execute(f"INSERT INTO {tabla} ({nombre_col}) VALUES (%s) RETURNING {id_col}", (v,))
                id_filtro = cur.fetchone()[0]

            cur.execute(f"INSERT INTO {tabla_intermedia} (id_producto, {id_col}) VALUES (%s, %s)", (id_producto, id_filtro))

def procesar_productos(df, cur):
    for _, row in df.iterrows():
        # Obtenemos los valores de las columnas, asignando None (NULL en SQL) si son NaN
        nombre = row['nombre'] if pd.notna(row['nombre']) else None
        descripcion = row['descripcion'] if pd.notna(row['descripcion']) else None
        marca = row['marca']
        categoria = row['categoria']
        subcategoria = row['subcategoria']
        precio = row['precio'] if pd.notna(row['precio']) else None
        numero_valoraciones = row['numero_valoraciones']
        num_variaciones = row['num_variaciones']
        valoracion = row['valoracion']
        fecha_extraccion = row['fecha_extraccion']
        efecto_sombra = row.get('efecto_sombra')
        textura = row.get('textura')
        formato = row.get('formato')
        responsabilidad = row.get('responsabilidad')
        efecto_labios = row.get('efecto_labios')
        efecto_mascara = row.get('efecto_mascara')
        formulacion = row.get('formulacion')
        tipo_piel = row.get('tipo_piel')
        cobertura = row.get('cobertura')
        acabado = row.get('acabado')

        cur.execute("SELECT id_producto FROM productos WHERE nombre = %s", (nombre,))
        producto_result = cur.fetchone()

        if producto_result:
            id_producto = producto_result[0]
            insertar_historico(cur, id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones)
            print(f"Histórico actualizado para producto existente: {nombre}")
        else:
            id_marca = obtener_o_insertar_id(cur, "id_marca", "marcas", "nombre_marca", marca) if pd.notna(marca) else None
            id_categoria = obtener_o_insertar_id(cur, "id_categoria", "categorias", "nombre_categoria", categoria) if pd.notna(categoria) else None
            id_subcategoria = obtener_o_insertar_id(cur, "id_subcategoria", "subcategorias", "nombre_subcategoria", subcategoria) if pd.notna(subcategoria) else None
            id_producto = insertar_producto(cur, nombre, descripcion, id_marca, id_categoria, id_subcategoria)
            insertar_historico(cur, id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, num_variaciones)
            insertar_filtro(cur, efecto_labios, "id_efecto_labios", "efectos_labios", "nombre_efecto", id_producto, "producto_efecto_labios")
            insertar_filtro(cur, efecto_sombra, "id_efecto_sombra", "efectos_sombra", "nombre_efecto", id_producto, "producto_efecto_sombra")
            insertar_filtro(cur, textura, "id_textura", "texturas", "nombre_textura", id_producto, "producto_textura")
            insertar_filtro(cur, formato, "id_formato", "formatos", "nombre_formato", id_producto, "producto_formato")  
            insertar_filtro(cur, responsabilidad, "id_responsabilidad", "responsabilidades", "nombre_responsabilidad", id_producto, "producto_responsabilidad")
            insertar_filtro(cur, efecto_mascara, "id_efecto_mascara", "efectos_mascara", "nombre_efecto", id_producto, "producto_efecto_mascara")
            insertar_filtro(cur, formulacion, "id_formulacion", "formulaciones", "nombre_formulacion", id_producto, "producto_formulacion")
            insertar_filtro(cur, tipo_piel, "id_tipo_piel", "tipos_piel", "nombre_tipo_piel", id_producto, "producto_tipo_piel")
            insertar_filtro(cur, cobertura, "id_cobertura", "coberturas", "nombre_cobertura", id_producto, "producto_cobertura")    
            insertar_filtro(cur, acabado, "id_acabado", "acabados", "nombre_acabado", id_producto, "producto_acabado")

def cargar_datos_sephora():
    df = pd.read_csv("../data/productos_maquillaje.csv")
    conn, cur = conectar_bd()

    procesar_productos(df, cur)  
    conn.commit()  

    cur.close()
    conn.close()
    print("Proceso finalizado ✅")

In [33]:
cargar_datos_sephora()

Histórico actualizado para producto existente: Beautyblender
Histórico actualizado para producto existente: Green Flash
Histórico actualizado para producto existente: Green
Histórico actualizado para producto existente: Color Twister
Histórico actualizado para producto existente: Ribbon Wrapped Lash
Histórico actualizado para producto existente: Revealer Concealer
Histórico actualizado para producto existente: Brush
Histórico actualizado para producto existente: Brush
Histórico actualizado para producto existente: BADgal Bounce
Histórico actualizado para producto existente: HD Skin Perfecting Pressed Powder
Histórico actualizado para producto existente: Dior Forever Nude Bronze
Histórico actualizado para producto existente: Essential Drip
Histórico actualizado para producto existente: Essential Drip
Histórico actualizado para producto existente: Colorful Sombra de Ojos
Histórico actualizado para producto existente: Colorful Sombra de Ojos
Histórico actualizado para producto existente: 

In [16]:
df = pd.read_csv('../data/productos_maquillaje.csv')
df.head()

,Unnamed: 0,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,...,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,formulacion,tipo_piel,cobertura,acabado,textura
0,0,Rostro,Base de maquillaje,Fenty Beauty,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,40.00,73,4.7,50,...,Standard,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,Media,Natural,Liquido
1,1,Rostro,Base de maquillaje,Haus Labs,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,49.99,35,4.5,51,...,"Standard, Frasco","Vegano, Clean at Sephora",NaN,NaN,NaN,NaN,Todo tipo de pieles,Media,Natural,Liquido
2,2,Rostro,Antiojeras & corrector,Haus Labs,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,34.99,20,4.1,30,...,Standard,"Vegano, Clean at Sephora",NaN,NaN,NaN,NaN,Todo tipo de pieles,Media,Natural,Liquido
3,3,Labios,Gloss/ Brillo labial,Haus Labs,PhD Hybrid Lip Oil,Aceite híbrido para labios,25.99,7,4.3,5,...,NaN,"Vegano, Clean at Sephora",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Rostro,Iluminador,Haus Labs,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,46.99,8,4.8,7,...,NaN,"Vegano, Clean at Sephora",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df[df['valoracion'].isna()]

,Unnamed: 0,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,...,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,formulacion,tipo_piel,cobertura,acabado,textura
